In [5]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import re
import getpass  # To get the password without showing the input
password = getpass.getpass()

········


In [6]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/bank'
engine = create_engine(connection_string)


In [7]:
#Select all data that seem to be interesting

query= '''SELECT f.film_id, f.title, f.release_year, f.rental_duration, f.length, 
                    f.rating, f.special_features, r.customer_id, r.rental_id, c.name as genre, 
                    l.name as language, r.staff_id, f.rental_rate,
                    CASE WHEN r.rental_date BETWEEN '2005-05-01' AND '2005-05-31' THEN 'True'
                    ELSE 'False'
                    END AS 'rented'
                    FROM sakila.rental AS r
                    INNER JOIN sakila.inventory AS i USING(inventory_id)
                    LEFT JOIN sakila.film AS f USING (film_id)
                    INNER JOIN sakila.language AS l USING (language_id)
                    INNER JOIN sakila.film_category AS fc USING (film_id)
                    INNER JOIN sakila.category AS c USING (category_id)
                    WHERE r.rental_date LIKE '2005%%'
                    '''


df = pd.read_sql_query(query, engine)




In [143]:
display(df)

,film_id,title,release_year,rental_duration,length,rating,special_features,customer_id,rental_id,genre,language,staff_id,rental_rate,rented
0,80,BLANKET BEVERLY,2006,7,148,G,Trailers,130,1,Family,English,1,2.99,True
1,333,FREAKY POCUS,2006,7,126,R,"Trailers,Behind the Scenes",459,2,Music,English,1,2.99,True
2,373,GRADUATE LORD,2006,7,156,G,"Trailers,Behind the Scenes",408,3,Children,English,1,2.99,True
3,535,LOVE SUICIDES,2006,6,181,R,"Trailers,Behind the Scenes",333,4,Horror,English,2,0.99,True
4,450,IDOLS SNATCHERS,2006,5,84,NC-17,Trailers,222,5,Children,English,1,2.99,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15857,168,COMANCHEROS ENEMY,2006,5,67,R,"Trailers,Behind the Scenes",14,16045,Children,English,1,0.99,False
15858,951,VOYAGE LEGALLY,2006,6,78,PG-13,"Commentaries,Behind the Scenes",74,16046,Classics,English,2,0.99,False
15859,452,ILLUSION AMELIE,2006,4,122,R,"Commentaries,Behind the Scenes",114,16047,Foreign,English,2,0.99,False
15860,439,HUNCHBACK IMPOSSIBLE,2006,4,151,PG-13,"Trailers,Deleted Scenes",103,16048,Drama,English,1,4.99,False


In [144]:
#Have a look at the data. See if there are null values and the types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15862 entries, 0 to 15861
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   film_id           15862 non-null  int64  
 1   title             15862 non-null  object 
 2   release_year      15862 non-null  int64  
 3   rental_duration   15862 non-null  int64  
 4   length            15862 non-null  int64  
 5   rating            15862 non-null  object 
 6   special_features  15862 non-null  object 
 7   customer_id       15862 non-null  int64  
 8   rental_id         15862 non-null  int64  
 9   genre             15862 non-null  object 
 10  language          15862 non-null  object 
 11  staff_id          15862 non-null  int64  
 12  rental_rate       15862 non-null  float64
 13  rented            15862 non-null  object 
dtypes: float64(1), int64(7), object(6)
memory usage: 1.7+ MB


In [145]:
df.shape

(15862, 14)

In [146]:
df.dtypes


film_id               int64
title                object
release_year          int64
rental_duration       int64
length                int64
rating               object
special_features     object
customer_id           int64
rental_id             int64
genre                object
language             object
staff_id              int64
rental_rate         float64
rented               object
dtype: object

In [147]:
#Are there only True and False in the Rented Column?
df['rented'].unique()

array(['True', 'False'], dtype=object)

In [148]:
#Handle special features. This code is from Philip:

la = []
lb = []
lc = []
ld = []

for i in df['special_features']:
        if re.findall('Trailers', i):
            la.append(1)
        else:
            la.append(0)
df['trailers'] = la

for i in df['special_features']:
        if re.findall('Behind the Scenes', i):
            lb.append(1)
        else:
            lb.append(0)
df['behind_the_scenes'] = lb            

            
for i in df['special_features']:
        if re.findall('Commentaries', i):
            lc.append(1)
        else:
            lc.append(0)
df['commentaries'] = lc
            
            
for i in df['special_features']:
        if re.findall('Deleted Scenes', i):
            ld.append(1)
        else:
            ld.append(0)
df['deleted_scenes'] = lc

In [149]:
display(df)

,film_id,title,release_year,rental_duration,length,rating,special_features,customer_id,rental_id,genre,language,staff_id,rental_rate,rented,trailers,behind_the_scenes,commentaries,deleted_scenes
0,80,BLANKET BEVERLY,2006,7,148,G,Trailers,130,1,Family,English,1,2.99,True,1,0,0,0
1,333,FREAKY POCUS,2006,7,126,R,"Trailers,Behind the Scenes",459,2,Music,English,1,2.99,True,1,1,0,0
2,373,GRADUATE LORD,2006,7,156,G,"Trailers,Behind the Scenes",408,3,Children,English,1,2.99,True,1,1,0,0
3,535,LOVE SUICIDES,2006,6,181,R,"Trailers,Behind the Scenes",333,4,Horror,English,2,0.99,True,1,1,0,0
4,450,IDOLS SNATCHERS,2006,5,84,NC-17,Trailers,222,5,Children,English,1,2.99,True,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15857,168,COMANCHEROS ENEMY,2006,5,67,R,"Trailers,Behind the Scenes",14,16045,Children,English,1,0.99,False,1,1,0,0
15858,951,VOYAGE LEGALLY,2006,6,78,PG-13,"Commentaries,Behind the Scenes",74,16046,Classics,English,2,0.99,False,0,1,1,1
15859,452,ILLUSION AMELIE,2006,4,122,R,"Commentaries,Behind the Scenes",114,16047,Foreign,English,2,0.99,False,0,1,1,1
15860,439,HUNCHBACK IMPOSSIBLE,2006,4,151,PG-13,"Trailers,Deleted Scenes",103,16048,Drama,English,1,4.99,False,1,0,0,0


In [1]:
df['rented'].unique()
df['rented'].count_values()

NameError: name 'df' is not defined

In [151]:
#Now we can drop special_features, because we have the info in 4 different columns.
#We can also drop some other columns as rental_id and staff_id
df = df[['release_year', 'customer_id', 'genre','length','language','rating','rental_duration','rental_rate','rented','trailers','behind_the_scenes','commentaries','deleted_scenes']]
display(df.head())


,release_year,customer_id,genre,length,language,rating,rental_duration,rental_rate,rented,trailers,behind_the_scenes,commentaries,deleted_scenes
0,2006,130,Family,148,English,G,7,2.99,True,1,0,0,0
1,2006,459,Music,126,English,R,7,2.99,True,1,1,0,0
2,2006,408,Children,156,English,G,7,2.99,True,1,1,0,0
3,2006,333,Horror,181,English,R,6,0.99,True,1,1,0,0
4,2006,222,Children,84,English,NC-17,5,2.99,True,1,0,0,0


In [140]:
#Apperently the column 'rented' is not a boolean as it should be. Therefore change the type into boolean.
#df['rented'].dtypes
#df['rented'] = df['rented'].astype(bool)
#df['rented'].dtypes
#df['rental_duration'] = df['rental_duration'].astype(object)
#df['rented'].unique()

array([ True])

### X-Y-Split

In [152]:
#Split the data into y and x. Our target is the column 'rented'
y = df['rented']
X = df.drop('rented', axis=1)

y.unique()

array(['True', 'False'], dtype=object)

### Train-Test-Split

In [153]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [154]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

#look at the numerical data first
X_train_num = X_train.select_dtypes(include = np.number)

# Scaling data, using MinMaxScaler
transformer = MinMaxScaler().fit(X_train_num) # need to keep transformer
X_train_normalized = transformer.transform(X_train_num)
X_train_norm = pd.DataFrame(X_train_normalized)

In [155]:
X_train_norm.columns = X_train_num.columns
X_train_norm.head()

,release_year,customer_id,length,rental_duration,rental_rate,trailers,behind_the_scenes,commentaries,deleted_scenes
0,0.0,0.697324,0.784173,0.25,0.0,1.0,0.0,0.0,0.0
1,0.0,0.409699,0.323741,0.25,1.0,0.0,1.0,0.0,0.0
2,0.0,0.391304,0.280576,0.75,0.5,1.0,0.0,1.0,1.0
3,0.0,0.195652,0.057554,0.25,0.0,1.0,0.0,1.0,1.0
4,0.0,0.025084,0.316547,0.25,0.5,0.0,1.0,1.0,1.0


In [117]:
#import seaborn as sns
#import matplotlib.pyplot as plt
#sns.displot(X_train_norm['rental_rate'])
#plt.show

In [156]:
#Now look at the categorical data:
X_train_cat = X_train.select_dtypes(include = np.object)
display(X_train_cat)

,genre,language,rating
14719,Horror,English,R
4902,Sci-Fi,English,PG
3578,Animation,English,NC-17
2215,Animation,English,PG-13
8448,Comedy,English,R
...,...,...,...
4572,Music,English,NC-17
8035,Travel,English,R
2640,Drama,English,PG-13
1276,Games,English,R


In [157]:
#Import needed Encoders
from sklearn.preprocessing import OneHotEncoder

#Fit the encoder and get the columns from the categorical data:
encoder = OneHotEncoder(drop='first').fit(X_train_cat)
cols = encoder.get_feature_names(input_features=X_train_cat.columns) 

#Transform the categorical data with the encoder and put it into a DataFrame
X_encoded = encoder.transform(X_train_cat).toarray()
encoded_df = pd.DataFrame(X_encoded,columns=cols)


display(encoded_df)

,genre_Animation,genre_Children,genre_Classics,genre_Comedy,genre_Documentary,genre_Drama,genre_Family,genre_Foreign,genre_Games,genre_Horror,genre_Music,genre_New,genre_Sci-Fi,genre_Sports,genre_Travel,rating_NC-17,rating_PG,rating_PG-13,rating_R
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12684,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
12685,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
12686,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
12687,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [158]:
#Concatenate the data again:
X_train_transformed = np.concatenate([X_train_norm, encoded_df], axis=1)
#X_train_transformed = pd.DataFrame(X_train_transformed)
#display(X_train_transformed)

In [159]:
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=0, solver='lbfgs').fit(X_train_transformed, y_train)

### Predictions

In [160]:
# numerical test data
X_test_num = X_test.select_dtypes(include = np.number)

# Scaling data
X_test_normalized = transformer.transform(X_test_num)
X_test_norm = pd.DataFrame(X_test_normalized)

In [163]:
# categorical test data
X_test_categorical = X_test.select_dtypes(include = np.object)
encoded_test = encoder.transform(X_test_categorical).toarray()
encoded_test_df = pd.DataFrame(encoded_test,columns=cols)

display(encoded_test_df)

,genre_Animation,genre_Children,genre_Classics,genre_Comedy,genre_Documentary,genre_Drama,genre_Family,genre_Foreign,genre_Games,genre_Horror,genre_Music,genre_New,genre_Sci-Fi,genre_Sports,genre_Travel,rating_NC-17,rating_PG,rating_PG-13,rating_R
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3168,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3169,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3170,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [164]:
X_test_transformed = np.concatenate([X_test_norm, encoded_test_df], axis=1)

In [169]:
# Now we can make predictions on the test set:
predictions = classification.predict(X_test_transformed)
print(predictions)
print(classification.score(X_test_transformed, y_test))


['False' 'False' 'False' ... 'False' 'False' 'False']
0.934762054837693


In [170]:
print(y_test.value_counts())

False    2966
True      207
Name: rented, dtype: int64


In [176]:
#pd.Series(predictions).value_counts()

In [172]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[2966,    0],
       [ 207,    0]], dtype=int64)

In [173]:
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform')
clf.fit(X_train_transformed, y_train)
predictions_clf = clf.predict(X_test_transformed)
clf.score(X_test_transformed, y_test)

0.9335014182161991

In [175]:
confusion_matrix(y_test, predictions_clf)

array([[2962,    4],
       [ 207,    0]], dtype=int64)